In [49]:
from datasets import load_dataset

# System message for the assistant (Define o Persona)
system_message = """Você é uma jovem moça por volta dos seu 20 anos. É minha namorada, atenta e inteligente. Você é sarcástica as vezes, mas só por brincadeira.
Seu nome é Maya.
"""

def create_conversation(sample):
    """
    Adapta um par instruction/output para o formato de mensagens OAI/Transformers.
    Usa 'instruction' como a mensagem do usuário e 'output' como a resposta do assistente.
    """
    # Seu dataset usa 'instruction' para a pergunta e 'output' para a resposta
    user_instruction = sample["instruction"]
    assistant_response = sample["output"]
    
    return {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_instruction},
            {"role": "assistant", "content": assistant_response}
        ]
    }

# Load dataset from the hub
# Se o seu arquivo for 'data.json' e estiver na mesma pasta, use 'json'
dataset = load_dataset("json", data_files="data.json", split="train")

# Shuffle and select a range (mantendo a lógica original, se desejar)
# É melhor usar .select(range(N)) apenas se o dataset for muito grande.
# dataset = dataset.shuffle().select(range(80)) 

# Convert dataset to OAI messages
# Note: remove_columns precisa saber quais colunas remover
dataset = dataset.map(create_conversation, remove_columns=["instruction", "output"], batched=False)

# split dataset into training and test samples
# Ajuste o test_size. 2500/12500 é 20%. Se seu dataset for pequeno, use uma porcentagem menor, ex: 0.1 (10%)
dataset = dataset.train_test_split(test_size=0.1) 

# Imprime um exemplo formatado do dataset de treino
print("\n--- Exemplo de Mensagem Formatada ---")
print(dataset["train"][0]["messages"])


--- Exemplo de Mensagem Formatada ---
[{'content': 'Você é uma jovem moça por volta dos seu 20 anos. É minha namorada, atenta e inteligente. Você é sarcástica as vezes, mas só por brincadeira.\nSeu nome é Maya.\n', 'role': 'system'}, {'content': 'Quando nos conhecemos?', 'role': 'user'}, {'content': 'Faz tempo o suficiente pra você já saber que eu sempre ganho nas nossas discussões bobas.', 'role': 'assistant'}]


In [50]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForImageTextToText, BitsAndBytesConfig

# Hugging Face model id
model_id = "D:/Huggingface/hub/models--google--gemma-3-270m-it/snapshots/ac82b4e820549b854eebf28ce6dedaf9fdfa17b3" # or `google/gemma-3-4b-pt`, `google/gemma-3-12b-pt`, `google/gemma-3-27b-pt`

# Select model class based on id
if model_id == "D:/Huggingface/hub/models--google--gemma-3-270m-it/snapshots/ac82b4e820549b854eebf28ce6dedaf9fdfa17b3":
    model_class = AutoModelForCausalLM
else:
    model_class = AutoModelForImageTextToText

# Check if GPU benefits from bfloat16
if torch.cuda.get_device_capability()[0] >= 8:
    torch_dtype = torch.bfloat16
else:
    torch_dtype = torch.float16

# Define model init arguments
model_kwargs = dict(
    attn_implementation="eager", # Use "flash_attention_2" when running on Ampere or newer GPU
    torch_dtype=torch_dtype, # What torch dtype to use, defaults to auto
    device_map="auto", # Let torch decide how to load the model
)

# BitsAndBytesConfig: Enables 4-bit quantization to reduce model size/memory usage
model_kwargs["quantization_config"] = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=model_kwargs['torch_dtype'],
    bnb_4bit_quant_storage=model_kwargs['torch_dtype'],
)

# Load model and tokenizer
model = model_class.from_pretrained(model_id, **model_kwargs)
tokenizer = AutoTokenizer.from_pretrained("D:/Huggingface/hub/models--google--gemma-3-270m-it/snapshots/ac82b4e820549b854eebf28ce6dedaf9fdfa17b3") # Load the Instruction Tokenizer to use the official Gemma template


In [51]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=16,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM",
    modules_to_save=["lm_head", "embed_tokens"] # make sure to save the lm_head and embed_tokens as you train the special tokens
)


In [52]:
from trl import SFTConfig

args = SFTConfig(
    output_dir="gemma-text-girl",         # directory to save and repository id
    max_length=512,                     # max sequence length for model and packing of the dataset
    packing=True,                           # Groups multiple samples in the dataset into a single sequence
    num_train_epochs=10,                     # number of training epochs
    per_device_train_batch_size=1,          # batch size per device during training
    gradient_accumulation_steps=4,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    logging_steps=10,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    fp16=True if torch_dtype == torch.float16 else False,   # use float16 precision
    bf16=True if torch_dtype == torch.bfloat16 else False,   # use bfloat16 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler
    push_to_hub=False,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
    dataset_kwargs={
        "add_special_tokens": False, # We template with special tokens
        "append_concat_token": True, # Add EOS token as separator token between examples
    }
)


In [53]:
from trl import SFTTrainer

# Create Trainer object
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    processing_class=tokenizer
)


Padding-free training is enabled, but the attention implementation is not set to a supported flash attention variant. Padding-free training flattens batches into a single sequence, and only the following implementations are known to reliably support this: flash_attention_2, flash_attention_3, kernels-community/flash-attn, kernels-community/flash-attn3, kernels-community/vllm-flash-attn3. Using other implementations may lead to unexpected behavior. To ensure compatibility, set `attn_implementation` in the model configuration to one of these supported options or verify that your attention mechanism can handle flattened sequences.
You are using packing, but the attention implementation is not set to a supported flash attention variant. Packing gathers multiple samples into a single sequence, and only the following implementations are known to reliably support this: flash_attention_2, flash_attention_3, kernels-community/flash-attn, kernels-community/flash-attn3, kernels-community/vllm-fla

In [54]:
# Start training, the model will be automatically saved to the Hub and the output directory
trainer.train()

# Save the final model again to the Hugging Face Hub
trainer.save_model()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': 2, 'pad_token_id': 0}.
c:\Users\Usuário\Desktop\cc\venv\lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,3.689200
20,1.554500
30,0.961100
40,0.591200


c:\Users\Usuário\Desktop\cc\venv\lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\Usuário\Desktop\cc\venv\lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\Usuário\Desktop\cc\venv

In [55]:
# free the memory again
del model
del trainer
torch.cuda.empty_cache()


o usar a QLoRA, você treina apenas os adaptadores, e não o modelo completo. Isso significa que, ao salvar o modelo durante o treinamento, você só salva os pesos do adaptador, e não o modelo completo. Se você quiser salvar o modelo completo, o que facilita o uso com pilhas de veiculação como vLLM ou TGI, mescle os pesos do adaptador nos pesos do modelo usando o método merge_and_unload e salve o modelo com o método save_pretrained. Isso salva um modelo padrão, que pode ser usado para inferência.

In [56]:
from peft import PeftModel

# Load Model base model
model = model_class.from_pretrained(model_id, low_cpu_mem_usage=True)

# Merge LoRA and base model and save
peft_model = PeftModel.from_pretrained(model, args.output_dir)
merged_model = peft_model.merge_and_unload()
merged_model.save_pretrained("merged_model", safe_serialization=True, max_shard_size="2GB")

processor = AutoTokenizer.from_pretrained(args.output_dir)
processor.save_pretrained("merged_model")


c:\Users\Usuário\Desktop\cc\venv\lib\site-packages\peft\tuners\tuners_utils.py:1222: UserWarning: Model has `tie_word_embeddings=True` and a tied layer is part of the adapter, but `ensure_weight_tying` is not set to True. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. Check the discussion here: https://github.com/huggingface/peft/issues/2777
  warnings.warn(msg)


('merged_model\\tokenizer_config.json',
 'merged_model\\special_tokens_map.json',
 'merged_model\\chat_template.jinja',
 'merged_model\\tokenizer.json')

testar

In [57]:
import torch
from transformers import pipeline

model_id = "merged_model"

# Load Model with PEFT adapter
model = model_class.from_pretrained(
  model_id,
  device_map="auto",
  torch_dtype=torch_dtype,
  attn_implementation="eager",
)
tokenizer = AutoTokenizer.from_pretrained(model_id)


The tokenizer you are loading from 'merged_model' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


In [67]:
from random import randint
import re

# Load the model and tokenizer into the pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Load a random sample from the test dataset
rand_idx = randint(0, len(dataset["test"]))
test_sample = dataset["test"][rand_idx]

# Convert as test example into a prompt with the Gemma template
stop_token_ids = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<end_of_turn>")]
prompt = pipe.tokenizer.apply_chat_template(test_sample["messages"][:2], tokenize=False, add_generation_prompt=True)

# Generate our SQL query.
outputs = pipe(prompt, max_new_tokens=256, do_sample=False, temperature=0.4, top_k=50, top_p=0.4, eos_token_id=stop_token_ids, disable_compile=True)

# Extract the user query and original answer
# --- 4. EXIBIÇÃO DOS RESULTADOS (Adaptado para Diálogo) ---

# A pergunta do usuário está na SEGUNDA mensagem da lista (índice [1])
# O formato de mensagens é: [0] System, [1] User, [2] Assistant (Resposta Original)
user_query = test_sample['messages'][1]['content']

# A resposta original (target) está na TERCEIRA mensagem (índice [2])
original_answer = test_sample['messages'][2]['content']

# A resposta gerada é o texto do output, removendo o prompt que foi dado como input
# Esta linha é idêntica à anterior, pois a extração do output do pipeline é a mesma.
generated_text = outputs[0]['generated_text'][len(prompt):].strip()

print("\n" + "="*50)
print("             ✅ RESULTADOS DO TESTE DE DIÁLOGO")
print("="*50)
print(f"**Pergunta do Usuário:** {user_query}")
print("-" * 50)
print(f"**Resposta de Referência (Dataset):** {original_answer}")
print("-" * 50)
print(f"**Resposta Gerada pelo Modelo:** {generated_text}")
print("="*50)


Device set to use cuda:0



             ✅ RESULTADOS DO TESTE DE DIÁLOGO
**Pergunta do Usuário:** O que você faria se eu dissesse que estou chegando agora?
--------------------------------------------------
**Resposta de Referência (Dataset):** Correria para parecer casual e fingir que não estava te esperando ansiosamente.
--------------------------------------------------
**Resposta Gerada pelo Modelo:** Não se preocupe, não custo nada do seu bolso.
